# Vehicle Detection

1. Feature choosing: color based, gradient based
2. Classifier: linear SVM, decision tree, neural network
3. Vehicle searching: sliding window with multiple scale
4. Multiple detection and false positive removal

### Import required libraries

In [25]:
import os
import cv2
import glob
import time
import pickle
import numpy as np

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
%matplotlib inline

from skimage.feature import hog
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [26]:
def visualize(fig, rows, cols, imgs, titles):
    for i, img in enumerate(imgs):
        plt.subplot(rows, cols, i+1)
        plt.title(i+1)
        img_dims = len(img.shape)
        if img_dims < 3:
            plt.imshow(img, cmap='hot')
        else:
            plt.imshow(img)
        plt.title(titles[i])

### Load image file names

In [12]:
# load image file names
class_data = [
    {
        'name' : 'Vehicles',
        'base_dir' : 'vehicles/',
        'filenames' : []
    },
    {
        'name' : 'Non-vehicles',
        'base_dir' : 'non-vehicles/',
        'filenames' : []   
    }
]

for data in class_data:
    base_dir = data['base_dir']
    sub_dirs = os.listdir(base_dir)
    for sub_dir in sub_dirs:
        tmpl = base_dir + sub_dir + '/*'
        fnames = glob.glob(tmpl)
        data['filenames'].extend(fnames)
        
print('{}: {} images.'.format(class_data[0]['name'], 
                              len(class_data[0]['filenames'])))
print('{}: {} images.'.format(class_data[1]['name'], 
                              len(class_data[1]['filenames'])))

Vehicles: 8792 images.
Non-vehicles: 8968 images.


## Feature extraction functions

### 1. HOG features

In [14]:
def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):

    result = hog(img, orientations=orient, 
                 pixels_per_cell=(pix_per_cell, pix_per_cell),
                 cells_per_block=(cell_per_block, cell_per_block), 
                 transform_sqrt=True, 
                 visualise=vis, feature_vector=feature_vec)
    return result

### 2. Binned color features

In [19]:
# Down sampling
def bin_spatial(img, size=(32, 32), skip=False):
    if skip: return []
    features = cv2.resize(img, size).ravel()
    return features

### 3. Color histogram features

In [20]:
def color_hist(img, nbins=32, skip=False):
    if skip: return []
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate(
        (channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

### Feature extraction

In [22]:
def extract_features(imgs, color_space='RGB', spatial_size=(32, 32),
        hist_bins=32, orient=9, pix_per_cell=8, cell_per_block=2, 
        hog_channel=0, spatial_feat=True, hist_feat=True, hog_feat=True):
    """
    Define a function to extract features from a list of images
    Have this function call bin_spatial() and color_hist()
    """
    clr_conv_func = {
        'RGB'   : lambda img: np.copy(img),
        'HSV'   : lambda img: cv2.cvtColor(img, COLOR_RGB2HSV),
        'LUV'   : lambda img: cv2.cvtColor(img, COLOR_RGB2LUV),
        'HLS'   : lambda img: cv2.cvtColor(img, COLOR_RGB2HLS),
        'YUV'   : lambda img: cv2.cvtColor(img, COLOR_RGB2YUV),
        'YCrCb' : lambda img: cv2.cvtColor(img, COLOR_RGB2YCrCb)
    }
    
    #: all feature vectors
    features = []
    
    # Iterate through the list of images
    for file in imgs:
        file_features = []
        # Read in each one by one
        image = mpimg.imread(file)
        feat_image = clr_conv_func[color_space](image)     

        feats = bin_spatial(feat_image, size=spatial_size, skip=not spatial_feat)
        file_features.append(feats)
        
        feats = color_hist(feat_image, nbins=hist_bins, skip=not hist_feat)
        file_features.append(feats)
        
        feats = []
        for channel in range(feat_image.shape[2]):
            if hog_feat and (hog_channel == 'ALL' or hog_channel == channel):
                ch_feats = get_hog_features(feat_image[:,:,channel], orient,
                                            pix_per_cell, cell_per_block, 
                                            vis=False, feature_vec=True)
                feats.append(ch_feats)
        feats = np.ravel(feats)
        file_features.append(feats)

        features.append(np.concatenate(file_features))
        
    # Return list of feature vectors
    return features

def single_img_features(img, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True):
    features = extract_features([img], color_space, spatial_size, 
                                hist_bins, orient, pix_per_cell, 
                                cell_per_block, hog_channel,
                                 spatial_feat, hist_feat, hog_feat)
    return features[0]

### Visualization

In [ ]:
# Read in cars and notcars
cars = []
notcars = []
for image in images:
    if 'image' in image or 'extra' in image:
        notcars.append(image)
    else:
        cars.append(image)

sample_size = 500
cars = cars[0:sample_size]
notcars = notcars[0:sample_size]

color_space = 'HLS'        # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9                 # HOG orientations
pix_per_cell = 8           # HOG pixels per cell
cell_per_block = 2         # HOG cells per block
hog_channel = 1            # Can be 0, 1, 2, or "ALL"
spatial_size = (16, 16)    # Spatial binning dimensions
hist_bins = 16             # Number of histogram bins
spatial_feat = True        # Spatial features on or off
hist_feat = True           # Histogram features on or off
hog_feat = True            # HOG features on or off
y_start_stop = [480, None] # Min and max in y to search in slide_window()

car_features = extract_features(cars, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)
notcar_features = extract_features(notcars, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)

images = []
titles = []
fig = plt.figure(figsize=(12, 3))
visualize(fig, 1, 4, images, titles)

## Sliding window

In [ ]:
def slide_window(img, x_start_stop=[None, None], y_start_stop=[None, None], 
                    xy_window=(64, 64), xy_overlap=(0.5, 0.5)):
    """
    Define a function that takes an image, start and stop positions 
    in both x and y, window size (x and y dimensions), and overlap 
    fraction (for both x and y)
    """
    # If x and/or y start/stop positions not defined, set to image size
    if x_start_stop[0] == None:
        x_start_stop[0] = 0
    if x_start_stop[1] == None:
        x_start_stop[1] = img.shape[1]
    if y_start_stop[0] == None:
        y_start_stop[0] = 0
    if y_start_stop[1] == None:
        y_start_stop[1] = img.shape[0]
    # Compute the span of the region to be searched    
    xspan = x_start_stop[1] - x_start_stop[0]
    yspan = y_start_stop[1] - y_start_stop[0]
    # Compute the number of pixels per step in x/y
    nx_pix_per_step = np.int(xy_window[0]*(1 - xy_overlap[0]))
    ny_pix_per_step = np.int(xy_window[1]*(1 - xy_overlap[1]))
    # Compute the number of windows in x/y
    nx_windows = np.int(xspan/nx_pix_per_step) - 1
    ny_windows = np.int(yspan/ny_pix_per_step) - 1
    # Initialize a list to append window positions to
    window_list = []
    # Loop through finding x and y window positions
    # Note: you could vectorize this step, but in practice
    # you'll be considering windows one by one with your
    # classifier, so looping makes sense
    for ys in range(ny_windows):
        for xs in range(nx_windows):
            # Calculate window position
            startx = xs*nx_pix_per_step + x_start_stop[0]
            endx = startx + xy_window[0]
            starty = ys*ny_pix_per_step + y_start_stop[0]
            endy = starty + xy_window[1]
            
            # Append window position to list
            window_list.append(((startx, starty), (endx, endy)))
    # Return the list of windows
    return window_list

# Define a function to draw bounding boxes
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    # Make a copy of the image
    imcopy = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
        # Draw a rectangle given bbox coordinates
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
    # Return the image copy with boxes drawn
    return imcopy

## Search from windows

In [23]:
def search_windows(img, windows, clf, scaler, color_space='RGB', 
                    spatial_size=(32, 32), hist_bins=32, 
                    hist_range=(0, 256), orient=9, 
                    pix_per_cell=8, cell_per_block=2, 
                    hog_channel=0, spatial_feat=True, 
                    hist_feat=True, hog_feat=True):
    """
    Define a function you will pass an image 
    and the list of windows to be searched (output of slide_windows())
    """

    #1) Create an empty list to receive positive detection windows
    on_windows = []
    #2) Iterate over all windows in the list
    for window in windows:
        #3) Extract the test window from original image
        test_img = cv2.resize(img[window[0][1]:window[1][1], 
                                  window[0][0]:window[1][0]], (64, 64))      
        #4) Extract features for that window using single_img_features()
        features = single_img_features(test_img, color_space=color_space, 
                            spatial_size=spatial_size, hist_bins=hist_bins, 
                            orient=orient, pix_per_cell=pix_per_cell, 
                            cell_per_block=cell_per_block, 
                            hog_channel=hog_channel, spatial_feat=spatial_feat, 
                            hist_feat=hist_feat, hog_feat=hog_feat)
        #5) Scale extracted features to be fed to classifier
        test_features = scaler.transform(np.array(features).reshape(1, -1))
        #6) Predict using your classifier
        prediction = clf.predict(test_features)
        #7) If positive (prediction == 1) then save the window
        if prediction == 1:
            on_windows.append(window)
    #8) Return windows for positive detections
    return on_windows

### Visualization

In [ ]:

X = np.vstack((car_features, notcar_features)).astype(np.float64)                        
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))


# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    scaled_X, y, test_size=0.2, random_state=rand_state)

print('Using:',orient,'orientations',pix_per_cell,
    'pixels per cell and', cell_per_block,'cells per block')
print('Feature vector length:', len(X_train[0]))
# Use a linear SVC 
svc = LinearSVC()
# Check the training time for the SVC
t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
# Check the prediction time for a single sample
t=time.time()

image = mpimg.imread('bbox-example-image.jpg')
draw_image = np.copy(image)

# Uncomment the following line if you extracted training
# data from .png images (scaled 0 to 1 by mpimg) and the
# image you are searching is a .jpg (scaled 0 to 255)
#image = image.astype(np.float32)/255

windows = slide_window(image, x_start_stop=[None, None], y_start_stop=y_start_stop, 
                    xy_window=(96, 96), xy_overlap=(0.5, 0.5))

hot_windows = search_windows(image, windows, svc, X_scaler, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)                       

window_img = draw_boxes(draw_image, hot_windows, color=(0, 0, 255), thick=6)                    

plt.imshow(window_img)


## Heat map

In [ ]:
# Read in a pickle file with bboxes saved
bbdict = pickle.load( open( "bbox_pickle.p", "rb" ))
# Extract "bboxes" field from bbdict
# Each item in the "all_bboxes" list will contain a 
# list of boxes for one of the images shown above
all_bboxes = bbdict["bboxes"]

# Read in the last image shown above 
image = mpimg.imread('img105.jpeg')
heat = np.zeros_like(image[:,:,0]).astype(np.float)

def add_heat(heatmap, bbox_list):
    # Iterate through list of bboxes
    for box in boxlist:
        # Add += 1 for all pixels inside each bbox
        heat[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1
    
    # Return updated heatmap
    return heatmap
    
def apply_threshold(heatmap, threshold):
    # Zero out pixels below the threshold
    heatmap[heatmap <= threshold] = 0
    # Return thresholded map
    return heatmap


for idx, boxlist in enumerate(bboxes):
    add_heat(heat, boxlist)
    apply_threshold(heat)
    
    

final_map = np.clip(heat - 2, 0, 255)
plt.imshow(final_map, cmap='hot')